In [1]:
import os
import pickle
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import List, Dict

import pandas as pd
import torch
import yaml
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset
from transformers import TrainingArguments, Trainer, IntervalStrategy, AutoModelForSequenceClassification, AutoTokenizer

In [2]:
os.environ['WANDB_DISABLED'] = 'true'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

---
# Model

In [3]:
@dataclass
class ModelConfig:
    model_name: str
    num_labels: int
    max_length: int
    truncation: bool
    padding: str
    return_tensors: str
    output_attentions: bool
    output_hidden_states: bool

    def __post_init__(self):
        self.num_labels = int(self.num_labels)
        self.max_length = int(self.max_length)
        self.truncation = bool(self.truncation)
        self.output_attentions = bool(self.output_attentions)
        self.output_hidden_states = bool(self.output_hidden_states)

    def create_tokenizer(self):
        tokenizer = AutoTokenizer.from_pretrained(self.model_name)

        return lambda input_tokens: tokenizer(
            input_tokens,
            padding=self.padding,
            max_length=self.max_length,
            truncation=self.truncation,
            return_tensors=self.return_tensors
        )

    def create_model(self):
        return AutoModelForSequenceClassification.from_pretrained(
            self.model_name,
            num_labels=self.num_labels,
            output_attentions=self.output_attentions,
            output_hidden_states=self.output_hidden_states,
        )

---
# Dataset transformations

In [4]:
class Transformation(ABC):

    def __init__(self, **kwargs):
        pass

    @abstractmethod
    def transform(self, dataset: pd.DataFrame) -> pd.DataFrame:
        pass


@dataclass
class TransformationConfig(yaml.YAMLObject):
    name: str
    args: Dict

In [5]:
class DummyTransformation(Transformation):

    def transform(self, dataset: pd.DataFrame) -> pd.DataFrame:
        return dataset

In [6]:
TRANSFORMATIONS = {
    'DUMMY': DummyTransformation,
}

---
# Train Set

In [7]:
@dataclass
class TrainSetConfig:
    path: str
    transformations: List[TransformationConfig]

    def __post_init__(self):
        self.transformations = [TransformationConfig(**t) for t in self.transformations]

In [8]:
@dataclass
class TrainingConfig:
    dataset: TrainSetConfig
    batch_size: int
    epochs: int
    learning_rate: float
    output_dir: str

    def __post_init__(self):
        self.dataset = TrainSetConfig(**self.dataset)
        self.batch_size = int(self.batch_size)
        self.epochs = int(self.epochs)
        self.learning_rate = float(self.learning_rate)

    def load_dataset(self) -> pd.DataFrame:
        df = pd.read_csv(self.dataset.path, header=None, names=['id', 'text', 'label'])

        for t in self.dataset.transformations:
            transformation = TRANSFORMATIONS[t.name](**t.args)
            df = transformation.transform(df)

        return df

---
# Metrics

In [39]:
# FIXME: solarski
# about the metrics - I'm not sure if we should have separate metrics for all steps (validation, test, control)
# I would rather go with single metrics config, that would be common for all those steps.
# Then we don't need strange test_set_select (etc.) methods, which are either way redundant since the metric are calculated
# during call to trainer.predict().
# Only thing that we are interested in is saving those metrics for test and control stage.

In [9]:
METRIC_FUNCTIONS = {
    'accuracy': accuracy_score,
}


@dataclass
class MetricsConfig:
    name: str

---
# Validation set

In [10]:
@dataclass
class ValidationSetConfig:
    path: str

In [11]:
@dataclass
class ValidationConfig:
    dataset: ValidationSetConfig
    batch_size: int
    metrics: List[MetricsConfig]
    metric_for_best_model: str

    def __post_init__(self):
        self.dataset = ValidationSetConfig(**self.dataset)
        self.batch_size = int(self.batch_size)
        self.metrics = [MetricsConfig(**m) for m in self.metrics]

    def load_dataset(self) -> pd.DataFrame:
        return pd.read_csv(self.dataset.path, header=None, names=['id', 'text', 'label'])

    # TODO: rename ???
    def validation_set_select(self, predictions) -> dict:
        predictions, true_labels = predictions[0], predictions[1]
        predictions = predictions[0].argmax(1)

        return {
            metric.name:
                METRIC_FUNCTIONS[metric.name](true_labels, predictions)
            for metric in self.metrics
        }

---
# Predictions

In [12]:
def save_predictions(object, predictions_filename):
    file = open(predictions_filename, 'wb')
    pickle.dump(object, file)
    file.close()

In [13]:
def load_predictions(predictions_filename):
    file = open(predictions_filename, 'rb')
    results_depickled = pickle.load(file)
    file.close()
    return results_depickled

---
# Test set

In [14]:
@dataclass
class TestSetConfig:
    path: str

In [15]:
@dataclass
class TestingConfig:
    dataset: TestSetConfig
    metrics: List[MetricsConfig]
    results_file: str

    def __post_init__(self):
        self.dataset = TestSetConfig(**self.dataset)
        self.metrics = [MetricsConfig(**m) for m in self.metrics]

    def load_dataset(self) -> pd.DataFrame:
        return pd.read_csv(self.dataset.path, header=None, names=['id', 'text', 'label'])

    # TODO: rename ???
    def test_set_select(self) -> dict:
        predictions = load_predictions(self.results_file)
        predictions, true_labels = predictions[0], predictions[1]
        predictions = predictions[0].argmax(1)

        return {
            metric.name:
                METRIC_FUNCTIONS[metric.name](true_labels, predictions)
            for metric in self.metrics
        }

---
# Control set

In [16]:
@dataclass
class ControlSetConfig:
    path: str

In [17]:
@dataclass
class ControlConfig:
    dataset: ControlSetConfig
    metrics: List[MetricsConfig]
    results_file: str

    def __post_init__(self):
        self.dataset = ControlSetConfig(**self.dataset)
        self.metrics = [MetricsConfig(**m) for m in self.metrics]

    def load_dataset(self) -> pd.DataFrame:
        return pd.read_csv(self.dataset.path, header=None, names=['id', 'text', 'label'])

    # TODO: rename ???
    def control_set_select(self) -> dict:
        predictions = load_predictions(self.results_file)
        predictions, true_labels = predictions[0], predictions[1]
        predictions = predictions[0].argmax(1)

        return {
            metric.name:
                METRIC_FUNCTIONS[metric.name](true_labels, predictions)
            for metric in self.metrics
        }

---
# Configuration

In [18]:
with open('./config.yaml', 'r') as stream:
    configuration = yaml.safe_load(stream)

In [19]:
model_config = ModelConfig(**configuration['model'])
model_config

ModelConfig(model_name='bert-base-cased', num_labels=12, max_length=256, truncation=True, padding='max_length', return_tensors='pt', output_attentions=True, output_hidden_states=True)

In [20]:
training_config = TrainingConfig(**configuration['training'])
training_config

TrainingConfig(dataset=TrainSetConfig(path='./data/thedeep.subset.train.txt', transformations=[TransformationConfig(name='DUMMY', args={})]), batch_size=2, epochs=1, learning_rate=0.001, output_dir='ClassificationBERT')

In [21]:
validation_config = ValidationConfig(**configuration['validation'])
validation_config

ValidationConfig(dataset=ValidationSetConfig(path='./data/thedeep.subset.validation.txt'), batch_size=2, metrics=[MetricsConfig(name='accuracy')], metric_for_best_model='accuracy')

In [22]:
test_config = TestingConfig(**configuration['testing'])
test_config

TestingConfig(dataset=TestSetConfig(path='./data/thedeep.subset.test.txt'), metrics=[MetricsConfig(name='accuracy')], results_file='./file_test_results.pickle')

In [23]:
control_config = ControlConfig(**configuration['control'])
control_config

ControlConfig(dataset=ControlSetConfig(path='./data/thedeep.subset.control.txt'), metrics=[MetricsConfig(name='accuracy')], results_file='./file_control_results.pickle')

---
# Notebook flow

In [24]:
tokenizer = model_config.create_tokenizer()
model = model_config.create_model()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [25]:
data = {
    'train': training_config.load_dataset()[:100],
    'validation': validation_config.load_dataset()[:10],
    'test': test_config.load_dataset()[:10],
    'control': control_config.load_dataset()[:10],
}

In [26]:
tokens = {
    dataset_type: tokenizer(dataset['text'].tolist())
    for dataset_type, dataset in data.items()
}

labels = {
    dataset_type: torch.tensor(dataset['label'].tolist())
    for dataset_type, dataset in data.items()
}

In [27]:
class TextDataset(Dataset):
    def __init__(self, tokens, labels: torch.Tensor):
        self.input_ids = tokens.input_ids
        self.attention_mask = tokens.attention_mask
        self.token_type_ids = tokens.token_type_ids
        self.y = labels

    def __len__(self):
        return len(self.y)

    def __getitem__(self, i):
        return {
            'input_ids': self.input_ids[i],
            'attention_mask': self.attention_mask[i],
            'token_type_ids': self.token_type_ids[i],
            'labels': self.y[i]
        }

In [28]:
datasets = {
    dataset_type: TextDataset(tokens[dataset_type], labels[dataset_type])
    for dataset_type in data.keys()
}

In [29]:
training_args = TrainingArguments(
    output_dir=training_config.output_dir,
    learning_rate=training_config.learning_rate,
    evaluation_strategy=IntervalStrategy.EPOCH,
    save_strategy=IntervalStrategy.EPOCH,
    logging_strategy=IntervalStrategy.EPOCH,
    per_device_train_batch_size=training_config.batch_size,
    per_device_eval_batch_size=validation_config.batch_size,
    load_best_model_at_end=True,
    metric_for_best_model=validation_config.metric_for_best_model,
    num_train_epochs=training_config.epochs,
)

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azure-identity 1.12.0 (c:\users\solarsan\appdata\roaming\python\python39\site-packages), Requirement.parse('azure-identity==1.7.0'), {'azureml-dataprep'}).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [30]:
trainer = Trainer(
    model=model,
    train_dataset=datasets['train'],
    eval_dataset=datasets['validation'],
    compute_metrics=validation_config.validation_set_select,
    args=training_args
)

In [31]:
trainer.train()

C:\Users\solarsan\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=50, training_loss=2.6041082763671874, metrics={'train_runtime': 18.4137, 'train_samples_per_second': 5.431, 'train_steps_per_second': 2.715, 'total_flos': 13156733952000.0, 'train_loss': 2.6041082763671874, 'epoch': 1.0})

In [41]:
test_result = trainer.predict(datasets['test'])
test_result.metrics

{'test_loss': 1.9329229593276978,
 'test_accuracy': 0.5,
 'test_runtime': 0.8011,
 'test_samples_per_second': 12.483,
 'test_steps_per_second': 6.241}

In [42]:
save_predictions(test_result, configuration['testing']['results_file'])

In [43]:
test_config.test_set_select()

{'accuracy': 0.5}

In [44]:
control_result = trainer.predict(datasets['control'])
control_result.metrics

{'test_loss': 2.724782705307007,
 'test_accuracy': 0.0,
 'test_runtime': 0.5437,
 'test_samples_per_second': 18.393,
 'test_steps_per_second': 9.196}

In [45]:
save_predictions(control_result, configuration['control']['results_file'])

In [46]:
control_config.control_set_select()

{'accuracy': 0.0}